In [1]:

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
!pip install -q -U keras-tuner
import keras_tuner as kt
     

In [2]:
(x_train, y_train) , (x_test,y_test) = mnist.load_data()
     

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [5]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

In [6]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 60-600
  hp_units = hp.Int('units', min_value=60, max_value=600, step=60)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [9]:
from keras.layers import Input, Flatten

input_layer = Input(shape=(28, 28))
flatten_layer = Flatten()(input_layer)


In [10]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3)

In [11]:
tuner.search(x_train, y_train, epochs=50, validation_split=0.2)

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 26s]
val_accuracy: 0.9692500233650208

Best val_accuracy So Far: 0.9794166684150696
Total elapsed time: 00h 05m 46s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 420 and the optimal learning rate for the optimizer
is 0.001.



In [12]:
hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.fit(x_train, y_train, epochs=50, validation_split=0.2)
     

Epoch 1/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8906 - loss: 0.3811 - val_accuracy: 0.9654 - val_loss: 0.1222
Epoch 2/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9704 - loss: 0.0977 - val_accuracy: 0.9747 - val_loss: 0.0880
Epoch 3/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9833 - loss: 0.0561 - val_accuracy: 0.9738 - val_loss: 0.0881
Epoch 4/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9885 - loss: 0.0374 - val_accuracy: 0.9747 - val_loss: 0.0858
Epoch 5/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9913 - loss: 0.0276 - val_accuracy: 0.9753 - val_loss: 0.0857
Epoch 6/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9938 - loss: 0.0207 - val_accuracy: 0.9792 - val_loss: 0.0796
Epoch 7/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9955 - loss: 0.0147 - val_accuracy: 0.9767 - val_loss: 0.0880
Epoch 8/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9952 - loss: 0.0143 - 

In [13]:
eval_result = hypermodel.evaluate(x_test, y_test)
print("[test loss, test accuracy]:", eval_result)
     

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 524us/step - accuracy: 0.9745 - loss: 0.2437
[test loss, test accuracy]: [0.19921085238456726, 0.9782999753952026]
